# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?
> This is a classification problem, because the mapped output is a discrete class label: PASS or FAIL.  If on the other hand, we mapped the input features to a continuous output value, such as a fail risk value, then this would be a regression problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call

n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1 # minus 1 because one column is the class label
n_passed = student_data[student_data.passed == 'yes'].shape[0]
n_failed = student_data[student_data.passed == 'no'].shape[0]
grad_rate = (n_passed / float(n_students)) * 100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [62]:
from sklearn.cross_validation import train_test_split
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Function wrapper for timing
# Mike: I stripped out timing methods from within train_classifier and predict_labels in favor of this wrapper.
import time
def timing(func):
    def wrap(*args):
        time1 = time.time()
        ret = func(*args)
        time2 = time.time()
        wrap.timing = time2 - time1
        return ret
    return wrap

In [8]:
# Train a model
@timing
def train_classifier(clf, X_train, y_train):
    clf.fit(X_train, y_train)

# Mike: I consolidated the TODO and other code in work below.
# TODO: Choose a model, import it and instantiate an object

In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

@timing
def predict_labels(clf, features, target):
    y_pred = clf.predict(features)
    return f1_score(target.values, y_pred, pos_label='yes')

In [10]:
# Train and predict using different training set sizes
# Mike: I removed print statements within this function since this information will show up in the results table

def train_predict(clf, X_train, y_train, X_test, y_test):
    
    # train
    train_classifier(clf, X_train, y_train)
    time_train = train_classifier.timing

    # predect on train data
    f1_train = predict_labels(clf, X_train, y_train)
    
    # predict on test data
    f1_test = predict_labels(clf, X_test, y_test)
    time_predict_test = predict_labels.timing
    
    # return train/predict results
    return pd.DataFrame(
        [[len(X_train), time_train, time_predict_test, f1_train, f1_test]], 
        columns=['Training set size', 'Training time (secs)', 'Prediction time (secs)', 'F1 score for training set', 'F1 score for test set']
    )

# Mike: I consolidated the TODO in work below
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [11]:
# helper function to display table with classification results
from IPython.display import display, HTML
def displayTable(clf, X_train, y_train, X_test, y_test):
    df = pd.DataFrame()
    training_set_sizes = [100, 200, 300]
    for num in training_set_sizes:
        results = train_predict(clf, X_train[:num], y_train[:num], X_test, y_test)
        df = df.append(results, ignore_index=True)
        
    df.set_index('Training set size', inplace=True)
    display(HTML(df.T.to_html(col_space='134px', float_format="{:.3f}".format)))

### Support Vector Machines
---
- What is the theoretical O(n) time & space complexity in terms of input size?
 - Train Time: Because this is a quadratic programming problem, training complexity is O(n^2) or O(n^3) depending on the model parameters.
 - Test Time: Prediction complexity is proportional number of support vectors. Generally this is O(nSV*d) where nSV is the number of support vectors and d is the dimenionsality or O(1) in terms of n
 - Space: For SVM we don't need to keep all the data around.  To represent the model we just need the support vectors, making space O(nSV) or O(1) in terms of n
- What are the general applications of this model?
 - SVMs can be used for classification, regression and outliers detection.
- What are its strengths and weaknesses?
 - Strength: Works well in high dimensional feature spaces.  The complexity of trained classifier depends on the number of support vectors rather than the dimensionality of the data.
 - Strength: Always has a unique solution or global optima.
 - Weakness: Do not work well with very large datasets since training time tends to be cubic with size of the dataset.
 - Weakness: Does not work well with noisy data.  Even a small number of mislabeled examples can reduce training time performance.
- Given what you know about the data so far, why did you choose this model to apply?
 - We are working with a fairly small data set, so therefore training time will still be small even with the exponential Big O.  In addition, it is feasible that we would only need to train once, so an expensive training computation might not be a big issue.
 - The data has a fairly high dimensional feature space, which SVM handles well since features that do not matter will not be a part of the support vectors.
 - Given we are working with skewed data, SVM may be a good choice because the parameters of an SVM allow adjusting for class weight.
 - If memory consumption is a large concern in our theoretical environment, the SVM will do well because it only needs to retain the support vectors and not the entire data set during testing.
 - Testing time is constant with respect to n so we satify the constraint on computation costs.

In [63]:
from sklearn import svm
displayTable(svm.SVC(), X_train, y_train, X_test, y_test)

Training set size,100,200,300
Training time (secs),0.001,0.003,0.005
Prediction time (secs),0.001,0.001,0.002
F1 score for training set,0.878,0.868,0.876
F1 score for test set,0.775,0.781,0.784


### Naive Bayes
---
- What is the theoretical O(n) time & space complexity in terms of input size?
 - Training time: O(n). We just need to calculate the likilhood function for the model parameters.
 - Testing time: O(1).  Testing time only depends on the number of hypothesis and dimensionality of the data.
 - Space: This is (1).  We just store the model we got during training.
- What are the general applications of this model?
 - Naive Bayes is a classification algorithm.
- What are its strengths and weaknesses?
  - Strength: Works well in high dimensional spaces due to conditional independence assumption.
  - Strength: Fast.  Training time is linear and testing time is constant with respect to n.
  - Strength: Works well with noisy data.
 - Weakness: Only practical in small hypothesis spaces.
 - Weakness: Cannot capture dependencies among variables.  For example AND or XOR relationships.
 - Weakness: Suffers from skewed data bias.  Unbalanaced class labels within the training data may cause one class to be prefered over another.
- Given what you know about the data so far, why did you choose this model to apply?
  - Given we care about execution time and memory for this particular problem, Naive bayes is an excellent fit, given training time is linear, testing time is constant and space is constant.
  - Given the few training exmaples, and relative high dimensionality compared to how many examples we have, other models might suffer greatly from the curse of dimensionality, but naive bayes does well due to the conditional idependence assumptions.
  - Given the data contains only two classes, the Naive Bayes weakness on large hypothesis spaces is not a concern.

In [64]:
from sklearn.naive_bayes import GaussianNB
displayTable(GaussianNB(), X_train, y_train, X_test, y_test)

Training set size,100,200,300
Training time (secs),0.001,0.001,0.001
Prediction time (secs),0.001,0.001,0.001
F1 score for training set,0.847,0.841,0.804
F1 score for test set,0.803,0.724,0.763


### Decision Trees
---
- What is the theoretical O(n) time & space complexity in terms of input size?
 - O(n · k^2) where k is the number of features
 
- What are the general applications of this model?
 - Classification and Regression
- What are its strengths and weaknesses?
 - Strength: Graphically easy to comprehend.
 - Strength: Does not require a lot of data preparation.  For example feature scaling is not needed for a decision tree since splitting only considers one feature at a time.
 - Strength: Able to handle non-linear relationships with the data.
 - Strength: Able to handle continuous and categorical data.
 - Strength: Implicitly does feature selection, since the highest information gain splits are at the top of the three.
 - Weakness: Prone to overfitting, especially with high dimensional data sets.
 - Weakness: Practical decicion tree algorithms cannot guarantee to return the globally optimal decision tree since only local optimas are considered at each node.
 - Weakness: Small changes inn data can provide vastly different trees.
- Given what you know about the data so far, why did you choose this model to apply?
 - Given we are working with skewed data, Decisions Trees may be a good choice because the parameters of an Decision Tree allow adjusting for class weight.

In [66]:
from sklearn.tree import DecisionTreeClassifier
displayTable(DecisionTreeClassifier(), X_train, y_train, X_test, y_test)

Training set size,100,200,300
Training time (secs),0.001,0.001,0.002
Prediction time (secs),0.001,0.001,0.001
F1 score for training set,1.000,1.000,1.000
F1 score for test set,0.655,0.742,0.633


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

> Support Vector Machines or SVM is my chosen model based on my experiments.  Models considered include Decision Trees, Naive Bayes, and SVM.  Criterion leading to my decision include how the model applies to the given data, limited resources, and performance.

> In order to choose the model that best applied to the data, first we must note key properties of the data.  Because we are working with high-dimensional space with relatively few data points to train on, we want to avoid models that will suffer from the curse of dimensionality.  Models such as K-NN would have difficulty finding similarity a high-dimensional space such as this and so were not considered for this problem.  Each of the models considered perform well in high-dimensional spaces.  Another consideration is the skewed nature of the data, with 67.09% of the data set passing.  We might want a model that allows for adjusting the weight of each class.  SVM provides a method for this within the models parameters.

> Next, we will look at resource consumption in terms of training time, testing time, and computer memory allocation.  Each contribute to cost, however some may contribute more significantly.  For example it is feasible to believe that we will be training once or infrequently compared to testing.  Because of this more importance is placed on a small testing time.

> Of the considered models, SVM does have a disadvantage in terms of training time given a Big-O that is exponential in terms of the number of students we train the model on.  However, given we are training on a small data set, the cost due to training is not largely impacted.  For testing time, SVM is constant with respect to the number of students we train the model on.  This is true for all models chosen for consideration.  In contrast to lazy learners which have to consider the entire data set when testing.  With a small data set, this might not be an issue, but as time goes on the data set will get larger and larger increasing testing time.  For this scenario, the constant testing time given by SVM is favorable.  For memory, each model considered does well here.  In contrast to lazy learners, we only need to store the learned parameters of the model instead of keeping around the entire data set.

> Finally we will look at performance in terms of F<sub>1</sub> Score which encompasses accuracy, recall, and precision.  With untrained models Naive Bayes performs similarly to SVM, however we are unable to adjust the parameters of the Naive Bayes model.  With tuning the SVM is able to get better permanence.

- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).

> To understand what SVMs do, first consider a simplified version.  Picture an graph with x, y coordinates.  Within the graph we have some data points of two difference classes.  The job of the SVM is to draw a line separating those two classes so that on one side of the line we have one class and on the other side we have the other.

> The line that SVM chooses to separate the two classes will be the line that creates the most margin between the line and each class subject to constraints.  For example the data may be messy in that some data points from one class might end up on the wrong side of the line, but it is still the line that minimizes how much we get wrong.

> Now instead of x, y coordinates, which is a 2-dimensional space, consider that each feature of the student (Internet Access, Free Time, etc) is a dimension.  In a more generalized version SVM is able to create a separating “line” in this higher dimensional space that separates the passing students from the failing students.

- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.

> Tuned parameters (see below for grid search):

> C=1

> class_weight={'yes': 1, 'no': 1}

> gamma=0.1

- What is the model's final F<sub>1</sub> score?

> 0.789473684211

In [72]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1_scorer = make_scorer(f1_score, pos_label='yes')
svr = svm.SVC(cache_size=4096)

clf = GridSearchCV(svr, scoring=f1_scorer, cv=10, param_grid={
    'C': [1, 10, 100, 1000],
    'gamma': ['auto', 0.1, 0.001, 0.0001],
    'class_weight': ['balanced', {'yes': 1, 'no': 1}]
})
clf.fit(X_train, y_train)

print clf.best_estimator_
print
print "Final F1 Score:", clf.score(X_test, y_test)

SVC(C=1, cache_size=4096, class_weight={'yes': 1, 'no': 1}, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Final F1 Score: 0.789473684211
